So far we've learnt how to scrape the web, and how to make a request for information from an API. Some websites make APIs even easier. Check out [RapidAPI](https://rapidapi.com/) they take care of writing most of the code for you.

We will use the [AeroDataBox API](https://rapidapi.com/aedbx-aedbx/api/aerodatabox/), which can retrieve all sorts of information about flights and airports. We will show you how to retrieve information about the airports, and then it's up to you to apply this, along with what you've already learnt this week, to **produce a function, which retrieves tomorrows flight information for the major airports in the cities you web scapped**.

In [ ]:
import pandas as pd
import requests

On the left hand side of the AeroDataBox API page, you'll see a list of options for information that you can retrieve:
> - Flights API
- Subsciption / PUSH API
- Airport API
- Aircraft API
- Healthcheck & Status API

1. We want to select `Airport API`

2. Then within Airport API we want to select `Search airports by location`

3. Now in the middle third you'll want to enter the `latitude` and `longitude` of any city to test... we chose Berlin: latitude 52.31 longitude 13.24. Next we changed the `radiusKM` to only 50km. And finally set `withFlightInfoOnly` to true, so it will only return airports which have flight data (scheduled or live) available.

4. On the right hand third of the screen you should see a block of code that looks pretty unfamiliar. This is because by default the code is probably set to *(Node.js) Axios*. However, we have the power to change this to familiar python. Select the dropdown box at the top of the code and select `python > requests`.

Now you can copy the code to your notebook and it should look a little something like the cell below:

In [ ]:
import requests

url = "https://aerodatabox.p.rapidapi.com/airports/iata/AMS"

headers = {
	"X-RapidAPI-Key": "YOUR-API-KEY",
	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers)

print(response.text)

{"icao":"EHAM","iata":"AMS","shortName":"Schiphol","fullName":"Amsterdam, Amsterdam Schiphol","municipalityName":"Amsterdam","location":{"lat":52.3086,"lon":4.763889},"country":{"code":"NL","name":"Netherlands"},"continent":{"code":"EU","name":"Europe"},"timeZone":"Europe/Amsterdam","urls":{"webSite":"http://www.schiphol.nl/","wikipedia":"https://en.wikipedia.org/wiki/Amsterdam_Schiphol_Airport","twitter":"http://twitter.com/Schiphol","googleMaps":"https://www.google.com/maps/@52.308601,4.763889,14z","liveAtc":"http://www.liveatc.net/search/?icao=EHAM","flightRadar":"https://www.flightradar24.com/52.31,4.76/14"}}


Let's view the response as `.json()` instead of `.text` so that it's easier to read

In [ ]:
response.json()

{'icao': 'EHAM',
 'iata': 'AMS',
 'shortName': 'Schiphol',
 'fullName': 'Amsterdam, Amsterdam Schiphol',
 'municipalityName': 'Amsterdam',
 'location': {'lat': 52.3086, 'lon': 4.763889},
 'country': {'code': 'NL', 'name': 'Netherlands'},
 'continent': {'code': 'EU', 'name': 'Europe'},
 'timeZone': 'Europe/Amsterdam',
 'urls': {'webSite': 'http://www.schiphol.nl/',
  'wikipedia': 'https://en.wikipedia.org/wiki/Amsterdam_Schiphol_Airport',
  'twitter': 'http://twitter.com/Schiphol',
  'googleMaps': 'https://www.google.com/maps/@52.308601,4.763889,14z',
  'liveAtc': 'http://www.liveatc.net/search/?icao=EHAM',
  'flightRadar': 'https://www.flightradar24.com/52.31,4.76/14'}}

We can now turn this into a dataframe using `.json_normalize()`

In [ ]:
pd.json_normalize(response.json())

,icao,iata,shortName,fullName,municipalityName,timeZone,location.lat,location.lon,country.code,country.name,continent.code,continent.name,urls.webSite,urls.wikipedia,urls.twitter,urls.googleMaps,urls.liveAtc,urls.flightRadar
0,EHAM,AMS,Schiphol,"Amsterdam, Amsterdam Schiphol",Amsterdam,Europe/Amsterdam,52.3086,4.763889,NL,Netherlands,EU,Europe,http://www.schiphol.nl/,https://en.wikipedia.org/wiki/Amsterdam_Schiph...,http://twitter.com/Schiphol,"https://www.google.com/maps/@52.308601,4.76388...",http://www.liveatc.net/search/?icao=EHAM,"https://www.flightradar24.com/52.31,4.76/14"


Let's now use this for the latitude and longitude of multiple cities

In [ ]:
def icao_airport_codes(latitudes, longitudes):

  #assert len(latitudes) == len(longitudes)
  
  list_for_df = []

  for i in range(len(latitudes)):

    url = f"https://aerodatabox.p.rapidapi.com/airports/search/location/{latitudes[i]}/{longitudes[i]}/km/100/16"

    querystring = {"withFlightInfoOnly":"true"}

    headers = {
      "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com",
      "X-RapidAPI-Key": "YOUR-API-KEY"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)

    list_for_df.append(pd.json_normalize(response.json()['items']))

  return pd.concat(list_for_df, ignore_index=True)

In [ ]:
icao_airport_codes([52.31, 52.37], [13.24, 4.89])

,icao,iata,name,shortName,municipalityName,countryCode,location.lat,location.lon
0,EDDB,BER,"Berlin, Berlin Brandenburg",Brandenburg,Berlin,DE,52.35139,13.493889
1,EHAM,AMS,"Amsterdam, Amsterdam Schiphol",Schiphol,Amsterdam,NL,52.30860,4.763889
2,EHRD,RTM,"Rotterdam, Rotterdam The Hague",The Hague,Rotterdam,NL,51.95690,4.437220


###### **Challenge 1:** Icao codes
If you use the above for all of your cities, you can create a DataFrame of all the airports and their associated `icao` codes. Perfect for a relational database.

###### **Challenge 2:** Arrivals information
Using what you have been shown above, plus the skills you've learnt in the last couple of days:
1. In `AeroDataBox API` use the `Flight API` > `FIDS/Schedules: Airport departures and arrivals by airport ICAO code` section
2. Fill out the parameters in the middle third and then copy the `python: requests` code from the right hand third
3. Explore the data you get back. What would be useful in your DataFrame and what can be excluded? Remember Gans wants to know about when people are arriving in the city
4. Make a DataFrame from the information you see as important
5. Condense everything you did above into a function that can take a list of ICAO codes as an input, and as an output gives you a DataFrame with the information for *tomorrows arrivals*

In [3]:
# your code here
import requests
import pandas as pd

headers = {
"X-RapidAPI-Key": "APIKEY",
"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}
def build_departures_df(data,icao):
    departures_df = pd.DataFrame()

    for f in range(len(data)):
        flight_num = data[f]['number']
        dep_date = data[f]['movement']['scheduledTimeLocal']
        destination = data[f]['movement']['airport']['name']
        status = data[f]['status']
        flight = pd.DataFrame({'depart_date':dep_date,'city_iso':[icao],'flight':flight_num,'destination':destination,'status':status})
        departures_df = pd.concat([departures_df,flight],axis=0)
    return departures_df

to_local_time = '2023-02-08T00:00'
from_local_time = '2023-02-08T08:00'
icao = 'EGLL' #london
flights_url = f'https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{to_local_time}/{from_local_time}'
response = requests.request("GET", flights_url, headers=headers)

flight_data = response.json()
print(response.status_code)
departures_df = build_departures_df(flight_data['departures'],icao)
display(departures_df)

403


KeyError: 'departures'

In [ ]:
def build_params(city):
    if city == 'Rome':
        return {
            "lat":41.8947,
            "lon":12.4839,
#             "appid":""
            "appid":""
        }
    if city == 'Paris':
        return {
            "lat":48.8534, 
            "lon":2.388,
            "appid":""
        }

In [5]:
# https://openweathermap.org/current
# url = "https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={long}&appid={token}
weather_df=pd.DataFrame()
current_weather = 'https://api.openweathermap.org/data/2.5/weather'

params = build_params('Rome')
response = requests.request("GET", current_weather, params=params)
try:
    weather_data =response.json()
    print(response.status_code)
    description_weather = weather_data['weather'][0]['description']
    dtime = weather_data['dt']
    time = pd.to_datetime(dtime, unit='s')
    # print(time)
    # print(data['main'])
    currtemp = weather_data['main']['temp']
    temp_min = weather_data['main']['temp_min']
    temp_max = weather_data['main']['temp_max']
    print(temp_min)


    current_df=pd.DataFrame({'date':time,'city':["Rome"],'city_iso':["LIRF"],'outlook':description_weather,'temp':currtemp,'temp_max':temp_max,'temp_min':temp_min})
    weather_df = pd.concat([weather_df,current_df],axis=0)
    display(weather_df)
except:
    print("Error from server: " + str(response.content))




200
281.7


,date,city,city_iso,outlook,temp,temp_max,temp_min
0,2023-02-09 12:56:27,Rome,LIRF,clear sky,282.7,284.08,281.7
